In [2]:
import os

In [5]:
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "smith-test"

# The below examples use the OpenAI API, so you will need
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")


content=''
content='Hello'
content='!'
content=' How'
content=' can'
content=' I'
content=' assist'
content=' you'
content=' today'
content='?'
content=''
